<a href="https://colab.research.google.com/github/shubamakabra/MagnusBox/blob/main/REMA_Q%26A_Processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Run this and then restart the runtime. There is some funk with the shapely installation

In [ ]:
!pip install -U google-cloud-aiplatform "shapely<2"
!pip install google-cloud-aiplatform==1.25.0
!pip install langchain==0.0.187
#!pip install xmltodict==0.13.0
#!pip install unstructured==0.7.0 # used by langchain
#!pip install pdf2image==1.16.3 #used by langchain
#!pip install requests==2.31.0
!pip install beautifulsoup4==4.12.2
!pip install chromadb
!pip install -U sentence-transformers

## ChromaDB vector database for context + VertexAI response generation

In [ ]:
from google.colab import auth as google_auth
google_auth.authenticate_user()

!gcloud config set project next-poc-400210
from google.colab import files

upoaded = files.upload()
!gcloud auth activate-service-account --key-file=next-poc-400210-640e2c5a3f5d.json
!gcloud config set account magnus@next-poc-400210.iam.gserviceaccount.com
!gsutil ls

PROJECT_ID = 'next-poc-400210'
REGION = 'us-central1'
BUCKET = 'gs://next-poc-bucket/REMA/vectors'
DISPLAY_NAME='REMA-document-answering'
TEXT_GENERATION_MODEL='text-bison@001'

Updated property [core/project].


Saving next-poc-400210-640e2c5a3f5d.json to next-poc-400210-640e2c5a3f5d.json
Activated service account credentials for: [magnus@next-poc-400210.iam.gserviceaccount.com]
Updated property [core/account].
gs://next-poc-bucket/


#Now we need to upload the JSON object we want to embed

In [ ]:
files.upload()

In [ ]:
import json

json_path = 'recipes.json'

with open(json_path, 'r') as file:
    recipesjson = file.read()

In [ ]:
import json
# This is likely reduntant, and might need a polish
formatted_json_string = json.dumps(json.loads(recipesjson))
json_object = json.loads(formatted_json_string)

In [ ]:
import chromadb
from sentence_transformers import SentenceTransformer
from langchain.embeddings import VertexAIEmbeddings
import pandas as pd

client = chromadb.PersistentClient(path="chroma_vectors")
collection = client.get_or_create_collection(name = 'REMA-document-embeddings', embedding_function = 'paraphrase-MiniLM-L3-v2')
model = SentenceTransformer('paraphrase-MiniLM-L3-v2')

In [ ]:
#client.delete_collection(name='REMA-document-embeddings')

In [ ]:
docs = []
embeds = []
metas = []
indexes = []
index = 0

for doc in json_object:

    docs.append(str(doc))
    doc_id = f"doc_{index}"
    metadata = {
        "document_id": doc_id,
        "cookMins": doc['cookMins'],
        "difficulty": doc['difficulty']
        }
    metas.append(metadata)

    embedding = model.encode(str(doc)).tolist()
    embeds.append(embedding)

    indexes.append(str(index))
    index = index + 1

In [ ]:
collection.add(
    documents = docs,
    metadatas = metas,
    embeddings = embeds,
    ids = indexes
)

In [ ]:
from vertexai.preview.language_models import TextGenerationModel
from langchain.vectorstores.matching_engine import MatchingEngine
from sentence_transformers import SentenceTransformer
from langchain.agents import Tool
from google.cloud import aiplatform
import os

aiplatform.init(project=PROJECT_ID, location=REGION)

def matching_engine_search(question):
    model = SentenceTransformer('paraphrase-MiniLM-L3-v2')
    input = model.encode(question).tolist()

    relevant_documentation = collection.query(
        query_embeddings=[input],
        n_results=8
    )
    context = relevant_documentation['documents']

    return str(context)

def query_the_machine(question):
  matching_engine_response=matching_engine_search(question)

  prompt = f"""
  Follow exactly these 3 steps:
  1. Read the context below and aggregate this data
  Context : {matching_engine_response}
  2. Create the answer from only the context
  3. Make a summary list of the ingredients
  4. Make a bulleted summary of the instructions
  5. Show the source for your answers
  User Question: {question}

  If you do not have any context or are unsure of the answer, reply that you do not know about this topic.
  """

  prompt2 = f"""
  Recipe Recommendation:
  You have a rich database of diverse recipes at your disposal. These recipes vary in flavors, origins, and dietary benefits.
  Your goal is to provide users with personalized meal suggestions based on their preferences and queries.
  Context: {matching_engine_response}
  User Query: {question}
  Your task is to generate a recipe suggestion that aligns with the user's needs.
  Each recipe you provide should include a brief summary of the dish, its cultural or regional origin,
  any dietary information (e.g., vegan, protein-rich), a list of ingredients with quantities, and step-by-step cooking instructions.
  You can respond with recipe suggestions and provide as much detail as possible, making the meal experience
  enjoyable and satisfying for the user. Feel free to engage in a conversation and explore a world of culinary delights. Let's get cooking!
  """


  model = TextGenerationModel.from_pretrained(TEXT_GENERATION_MODEL)
  response = model.predict(
              prompt2,
              temperature = 0.2,
              top_k = 40,
              top_p = 0.8,
              max_output_tokens = 1024,
  )

  print(f"Question: \n{question}")
  print(f"Response: \n{response.text}")

In [ ]:
question = "Suggest a meal that contains milk and meat"
query_the_machine(question)

Here is the question : 

Suggest a meal that contains milk and meat
Question: 
Suggest a meal that contains milk and meat
Response: 
Sure, here is a recipe for a creamy pasta dish that contains milk and meat:

**Ingredients:**

* 1 pound penne pasta
* 1 pound ground beef
* 1 onion, chopped
* 2 cloves garlic, minced
* 1 (28 ounce) can crushed tomatoes
* 1 (15 ounce) can tomato sauce
* 1 cup heavy cream
* 1/2 cup grated Parmesan cheese
* 1/2 teaspoon dried oregano
* 1/2 teaspoon salt
* 1/4 teaspoon black pepper

**Instructions:**

1. In a large pot of boiling salted water, cook the pasta according to the package directions.
2. While the pasta is cooking, brown the ground beef in a large skillet over medium heat. Drain the excess fat.
3. Add the onion and garlic to the skillet and cook until softened, about 5 minutes.
4. Stir in the crushed tomatoes, tomato sauce, heavy cream, Parmesan cheese, oregano, salt, and pepper. Bring to a simmer and cook for 15 minutes, or until the sauce has thi

In [ ]:
questions = ["", "", ""]
for q in questions:
  query_the_machine(q)

In [ ]:
!gsutil cp -r chroma_vectors gs://next-poc-bucket/REMA/vectors